#### Investigate the file

In [31]:
import pandas as pd
df = pd.read_csv('order-item-data.csv',sep='\t')

In [32]:
df.head()

,order_id,order_item_id,brand_city,brand_state,brand_country,order_grand_total,order_shipping_charge,order_created,event_created,event_expiration_date,order_item_quantity,order_item_unit_price,order_item_total_price,order_item_name,order_item_bounds,customer_billing_state,customer_billing_country,customer_shipping_state,customer_shipping_country
0,1797804,9118809,Auckland,NaN,NZ,39.5,0.0,2015-10-01 00:00:24,2015-09-25 23:17:54,NaN,1,39.5,39.5,Full Resolution - Approx 2-4MB (Digital Media),NaN,NaN,NaN,Auckland,NZ
1,1797805,9118810,Medford,OR,US,27.0,0.0,2015-10-01 00:00:52,2015-04-15 05:48:02,2015-09-30 07:00:00,1,15.0,15.0,(1) 8x10 (Headshot Prints),0.0,OR,US,OR,US
2,1797805,9118811,Medford,OR,US,27.0,0.0,2015-10-01 00:00:52,2015-04-15 05:48:02,2015-09-30 07:00:00,1,15.0,15.0,(1) 8x10 (Headshot Prints),0.0,OR,US,OR,US
3,1797806,9118812,Harrah,OK,US,9.0,0.0,2015-10-01 00:00:58,2015-09-29 08:25:36,NaN,1,3.0,3.0,Full Resolution (Digital Media),NaN,OK,US,OK,US
4,1797806,9118813,Harrah,OK,US,9.0,0.0,2015-10-01 00:00:58,2015-09-29 08:25:36,NaN,1,3.0,3.0,Full Resolution (Digital Media),NaN,OK,US,OK,US


Checking to see which columns are populated. 

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9998 entries, 0 to 9997
Data columns (total 19 columns):
order_id                     9998 non-null int64
order_item_id                9998 non-null int64
brand_city                   9940 non-null object
brand_state                  9062 non-null object
brand_country                9998 non-null object
order_grand_total            9998 non-null float64
order_shipping_charge        9998 non-null float64
order_created                9998 non-null object
event_created                9998 non-null object
event_expiration_date        7169 non-null object
order_item_quantity          9998 non-null int64
order_item_unit_price        9998 non-null float64
order_item_total_price       9998 non-null float64
order_item_name              9998 non-null object
order_item_bounds            7769 non-null float64
customer_billing_state       5807 non-null object
customer_billing_country     5850 non-null object
customer_shipping_state      9489 non-nul

#### Product Buckets
I noticed that each order item name had a category associated. This is always the last set of parenthesis in the item name. In order to extract that category I wrote a function to reverse the string, find the last set of parenthesis, extract the category, and reverse the string back to normal.

In [34]:
for i in list(df.order_item_name.unique()[0:10]):
    print (i)

Full Resolution - Approx 2-4MB (Digital Media)
(1) 8x10 (Headshot Prints)
Full Resolution (Digital Media)
2-5x7 (Packages)
Lustre (Prints)
Round Button 3 in. (Products)
Web Digitals (Digital Media)
Lustre - rounded corners (Prints)
5x7 Archival Print (Prints)
Set of Wallets (8/set) (Prints)


In [35]:
def reverse_string(s):
    new_s = ''
    for i in s:
        new_s = i + new_s
    return new_s

def get_item_category(s):
    reversed_s = reverse_string(s)
    start = reversed_s.find(')')
    end = reversed_s.find('(')
    
    item_category =  reverse_string(reversed_s[start:end+1])
    item_category = item_category.replace('(','').replace(')','')
    return item_category

df['item_category'] = df.order_item_name.apply(get_item_category)

#### Why doesn't the order_grand_total match the sum of the order's item's total price?
In the example below I would expect the order grand total to equal 30 not 27

In [36]:
df[df.order_id == 1797805]

,order_id,order_item_id,brand_city,brand_state,brand_country,order_grand_total,order_shipping_charge,order_created,event_created,event_expiration_date,order_item_quantity,order_item_unit_price,order_item_total_price,order_item_name,order_item_bounds,customer_billing_state,customer_billing_country,customer_shipping_state,customer_shipping_country,item_category
1,1797805,9118810,Medford,OR,US,27.0,0.0,2015-10-01 00:00:52,2015-04-15 05:48:02,2015-09-30 07:00:00,1,15.0,15.0,(1) 8x10 (Headshot Prints),0.0,OR,US,OR,US,Headshot Prints
2,1797805,9118811,Medford,OR,US,27.0,0.0,2015-10-01 00:00:52,2015-04-15 05:48:02,2015-09-30 07:00:00,1,15.0,15.0,(1) 8x10 (Headshot Prints),0.0,OR,US,OR,US,Headshot Prints


In this example there order has no total, but each item should cost 25. I would expect to see 125 as the order_grand_total

In [37]:
df[df.order_id == 1797811]

,order_id,order_item_id,brand_city,brand_state,brand_country,order_grand_total,order_shipping_charge,order_created,event_created,event_expiration_date,order_item_quantity,order_item_unit_price,order_item_total_price,order_item_name,order_item_bounds,customer_billing_state,customer_billing_country,customer_shipping_state,customer_shipping_country,item_category
24,1797811,9118833,Fayetteville,NC,US,0.0,0.0,2015-10-01 00:03:06,2015-09-08 14:59:15,2015-10-03 04:00:00,1,25.0,25.0,Full Resolution (Digital Media),NaN,NaN,NaN,NC,US,Digital Media
25,1797811,9118834,Fayetteville,NC,US,0.0,0.0,2015-10-01 00:03:06,2015-09-08 14:59:15,2015-10-03 04:00:00,1,25.0,25.0,Full Resolution (Digital Media),NaN,NaN,NaN,NC,US,Digital Media
26,1797811,9118835,Fayetteville,NC,US,0.0,0.0,2015-10-01 00:03:06,2015-09-08 14:59:15,2015-10-03 04:00:00,1,25.0,25.0,Full Resolution (Digital Media),NaN,NaN,NaN,NC,US,Digital Media
27,1797811,9118836,Fayetteville,NC,US,0.0,0.0,2015-10-01 00:03:06,2015-09-08 14:59:15,2015-10-03 04:00:00,1,25.0,25.0,Full Resolution (Digital Media),NaN,NaN,NaN,NC,US,Digital Media
28,1797811,9118837,Fayetteville,NC,US,0.0,0.0,2015-10-01 00:03:06,2015-09-08 14:59:15,2015-10-03 04:00:00,1,25.0,25.0,Full Resolution (Digital Media),NaN,NaN,NaN,NC,US,Digital Media


Looking at the data grouped by order_id will let me know how common this occurs

In [38]:
order_df = df.groupby(['order_id',
                       'order_created'], as_index=False)\
    .agg({'order_grand_total': 'mean',
          'order_shipping_charge':'mean',
          'order_item_total_price':'sum'})

In [39]:
from __future__ import division
unequal_orders = len(order_df[order_df.order_grand_total != order_df.order_item_total_price])  
total_orders = len(order_df)

print ("%s of all orders have an order grand total that does not match the sum of the item totals." 
       % "{:.2%}".format(unequal_orders/total_orders))

68.15% of all orders have an order grand total that does not match the sum of the item totals.


In [40]:
order_df.head()

,order_id,order_created,order_grand_total,order_shipping_charge,order_item_total_price
0,1797804,2015-10-01 00:00:24,39.50,0.0,39.5
1,1797805,2015-10-01 00:00:52,27.00,0.0,30.0
2,1797806,2015-10-01 00:00:58,9.00,0.0,9.0
3,1797807,2015-10-01 00:01:30,23.98,0.0,22.0
4,1797808,2015-10-01 00:01:37,20.00,0.0,20.0


#### Category Summary

In [41]:
df.columns

Index([u'order_id', u'order_item_id', u'brand_city', u'brand_state',
       u'brand_country', u'order_grand_total', u'order_shipping_charge',
       u'order_created', u'event_created', u'event_expiration_date',
       u'order_item_quantity', u'order_item_unit_price',
       u'order_item_total_price', u'order_item_name', u'order_item_bounds',
       u'customer_billing_state', u'customer_billing_country',
       u'customer_shipping_state', u'customer_shipping_country',
       u'item_category'],
      dtype='object')

In [42]:
category_df = df.groupby('item_category',as_index=False)\
        .agg({'order_id':pd.Series.nunique,
              'order_item_total_price':'sum',
              
             
             })\
        .sort_values('order_id',ascending=False)
        
category_df['spend_per_order'] = category_df['order_item_total_price'] / category_df['order_id'] 
category_df

,item_category,order_id,order_item_total_price,spend_per_order
75,Prints,869,69854.38,80.384787
30,Digital Media,416,37677.38,90.570625
79,Products,55,7502.13,136.402364
67,Packages,21,922.00,43.904762
14,Canvases,15,2932.97,195.531333
62,PRINTS,10,826.50,82.650000
25,Digital Files,10,1675.00,167.500000
0,,7,205.90,29.414286
54,Individual Prints,7,173.00,24.714286
29,Digital Images,7,1358.16,194.022857


#### Save files to csv for Tableau

In [43]:
df.to_csv('order-item-data-with-category.csv',index=False)
order_df.to_csv('order-data.csv',index=False)
category_df.to_csv('category-data.csv',index=False)